In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()

URM_train_list=[]
URM_validation_list=[]
evaluator_list=[]

In [3]:
for k in range(5):
    URM_train,URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage=0.8)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)
    evaluator_list.append(EvaluatorHoldout(URM_validation, cutoff_list=[10],verbose=False))

In [4]:
URM_combined_list=[]
for URM_i in URM_train_list:
    URM_combined_list.append(combine_matrices(ICM=ICM,URM=URM_i))

In [5]:
from bayes_opt import BayesianOptimization

In [6]:
rp3_recommenders=[]
slime_recommenders=[]
IALS_recommenders=[]
similarity_recommenders=[]
recommenders=[]


for i in range(len(URM_train_list)):
    
    rp3_recommenders.append(RP3betaRecommender(URM_train=URM_combined_list[i]))
    rp3_recommenders[i].fit(topK=181, 
                            alpha=0.5709402717259106, 
                            beta=0.3566066452521787, 
                            implicit=True, 
                            normalize_similarity=True)
    
    slime_recommenders.append(MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_combined_list[i]))
    slime_recommenders[i].fit(topK=420, 
                              l1_ratio=0.06061216785568925, 
                              alpha=0.00554982938879437, 
                              workers = 10)
    
    sim_slime = slime_recommenders[i].W_sparse
    sim_rp3 = rp3_recommenders[i].W_sparse
    
    similarity_recommenders.append(ItemKNNSimilarityHybridRecommender(URM_train=URM_combined_list[i],
                                                                     Similarity_1=sim_rp3,
                                                                     Similarity_2=sim_slime))
    


 21%|███████▉                              | 3752/18059 [01:53<04:04, 58.47it/s]


 40%|███████████████▎                      | 7304/18059 [03:38<03:24, 52.50it/s]


 60%|██████████████████████▏              | 10824/18059 [05:24<02:41, 44.66it/s]


 80%|█████████████████████████████▍       | 14376/18059 [07:11<01:24, 43.52it/s]


 99%|████████████████████████████████████▍| 17800/18059 [08:56<00:05, 44.85it/s]


...

 28%|██████████▋                           | 5096/18059 [02:52<05:23, 40.05it/s]


 47%|█████████████████▊                    | 8456/18059 [04:46<04:15, 37.65it/s]


 65%|████████████████████████▏            | 11816/18059 [06:42<05:18, 19.62it/s]


 84%|██████████████████████████████▉      | 15112/18059 [08:33<01:38, 29.99it/s]


100%|████████████████████████████████████▉| 18056/18059 [19:26<00:00, 15.48it/s]

100%|████████████████████████████████████▉| 18032/18059 [10:10<00:00, 66.36it/s]

In [20]:
ITEMCF_recommenders = []
sim_hybrid_itemcf_recommenders=[]

for i in range(len(URM_train_list)):
    
    ITEMCF_recommenders.append(ItemKNNCFRecommender(URM_train = URM_combined_list[i]))
    ITEMCF_recommenders[i].fit(topK=273, shrink=5.0, similarity='cosine')
    
    similarity_recommenders[i].fit(alpha=0.16115285930157153, topK=int(524.9285577442362))
    
    sim_ITEMCF =  ITEMCF_recommenders[i].W_sparse
    sim_hybrid = similarity_recommenders[i].W_sparse
    
    sim_hybrid_itemcf_recommenders.append(ItemKNNSimilarityHybridRecommender(URM_train=URM_combined_list[i],
                                                                     Similarity_1=sim_ITEMCF,
                                                                     Similarity_2=sim_hybrid))
    
tuning_params_itemcf = {
    "alpha":(0.0,1.0),
    "topK":(300,600)
}

def BO_function_itemcf(alpha,topK):
    MAP = 0
    k = 5
    cutoff=10
    for i in range(len(similarity_recommenders)):
        sim_hybrid_itemcf_recommenders[i].fit(topK=int(topK),alpha=alpha)
        result_df, _ = evaluator_list[i].evaluateRecommender(sim_hybrid_itemcf_recommenders[i])
        MAP += result_df.loc[cutoff]["MAP"]
        
    return MAP/k

optimizer_itemcf = BayesianOptimization(f=BO_function_itemcf,
                                pbounds=tuning_params_itemcf,
                                verbose=5,
                                random_state=5)

optimizer_itemcf.maximize(init_points=60,n_iter=70)

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1114.41 column/sec. Elapsed time 16.20 sec
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1044.69 column/sec. Elapsed time 17.29 sec
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1107.59 column/sec. Elapsed time 16.30 sec
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1096.09 column/sec. Elapsed time 16.48 sec
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1106.53 column/sec. Elapsed time 16.32 sec
|   iter    |  target   |   alpha   |   topK    |
-------------------------------------------------
|  1        |  0.2196   |  0.222    |  561.2    |
|  2        |  0.2203   |  0.2067   |  575.6    |
|  3        |  0.2134   |  0.4884   |  483.5    |
|  4        |  0.211    |  0.7659   |  455.5    |
|  5 

KeyboardInterrupt: 

In [7]:
tuning_params = {
    "alpha":(0.0,0.3),
    "topK":(450,600)
}

In [8]:

def BO_function(alpha,topK):
    MAP = 0
    k = 5
    cutoff=10
    for i in range(len(similarity_recommenders)):
        similarity_recommenders[i].fit(topK=int(topK),alpha=alpha)
        result_df, _ = evaluator_list[i].evaluateRecommender(similarity_recommenders[i])
        MAP += result_df.loc[cutoff]["MAP"]
        
    return MAP/k


In [9]:
optimizer = BayesianOptimization(f=BO_function,
                                pbounds=tuning_params,
                                verbose=5,
                                random_state=5)

optimizer.maximize(init_points=100,n_iter=100)


|   iter    |  target   |   alpha   |   topK    |
-------------------------------------------------



100%|████████████████████████████████████▉| 18056/18059 [10:22<00:00, 66.36it/s]

|  1        |  0.2471   |  0.0666   |  580.6    |
|  2        |  0.2471   |  0.06202  |  587.8    |


100%|████████████████████████████████████▉| 18056/18059 [13:08<00:00, 22.91it/s]


|  3        |  0.2479   |  0.1465   |  541.8    |
|  4        |  0.2485   |  0.2298   |  527.8    |
|  5        |  0.2473   |  0.08904  |  478.2    |
|  6        |  0.2466   |  0.02422  |  560.8    |
|  7        |  0.2477   |  0.1324   |  473.7    |
|  8        |  0.2487   |  0.264    |  491.1    |
|  9        |  0.2477   |  0.1243   |  494.4    |
|  10       |  0.2483   |  0.1886   |  537.0    |
|  11       |  0.2483   |  0.18     |  489.9    |
|  12       |  0.2473   |  0.08541  |  488.0    |
|  13       |  0.2474   |  0.09827  |  471.6    |
|  14       |  0.2469   |  0.04968  |  594.6    |
|  15       |  0.2486   |  0.2881   |  478.3    |
|  16       |  0.2464   |  0.007292 |  480.7    |
|  17       |  0.2485   |  0.21     |  566.9    |
|  18       |  0.2464   |  0.00688  |  536.6    |
|  19       |  0.2463   |  0.000492 |  527.3    |
|  20       |  0.2484   |  0.1919   |  597.8    |
|  21       |  0.2472   |  0.07773  |  570.4    |
|  22       |  0.2487   |  0.2611   |  588.4    |


|  103      |  0.2482   |  0.1725   |  587.0    |
|  104      |  0.2487   |  0.3      |  594.9    |
|  105      |  0.2479   |  0.1457   |  509.9    |
|  106      |  0.2487   |  0.2618   |  570.1    |
|  107      |  0.2483   |  0.1853   |  524.0    |
|  108      |  0.2487   |  0.293    |  524.2    |
|  109      |  0.2487   |  0.2823   |  509.7    |
|  110      |  0.2486   |  0.3      |  486.1    |
|  111      |  0.2484   |  0.2356   |  466.3    |
|  112      |  0.2478   |  0.1361   |  534.2    |
|  113      |  0.2487   |  0.2983   |  534.0    |
|  114      |  0.2483   |  0.1794   |  592.9    |
|  115      |  0.2487   |  0.2991   |  588.6    |
|  116      |  0.2475   |  0.09696  |  588.6    |
|  117      |  0.2476   |  0.1121   |  593.2    |
|  118      |  0.2486   |  0.2826   |  491.3    |
|  119      |  0.2475   |  0.1035   |  491.3    |
|  120      |  0.2487   |  0.2558   |  586.7    |
|  121      |  0.2487   |  0.2661   |  545.0    |
|  122      |  0.2479   |  0.1451   |  544.8    |


In [11]:
import json
with open("tuning_KFOLD/similarity_merging_SLIM_RP3/" + similarity_recommenders[0].RECOMMENDER_NAME + "2_logs.json", 'w') as json_file:
    json.dump(optimizer.max, json_file)

In [13]:
for i in range(len(URM_train_list)):
    IALS_recommenders.append(IALSRecommender(URM_train=URM_combined_list[i]))
    IALS_recommenders[i].fit(epochs=10, 
                             num_factors=42, 
                             alpha=0.7617528864750021, 
                             reg=8.926401306541349)
    
    similarity_recommenders[i].fit(topK=int(579.348017759496), alpha=0.05982163675463459)
    
    
    recommender = MergeTwoModelsByScores(URM_train=URM_combined_list[i],
                                         recommenders=[similarity_recommenders[i],IALS_recommenders[i]])
    recommenders.append(recommender)



IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 10.00 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 18.31 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 25.99 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 33.40 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 43.63 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 51.36 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.00 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.13 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.26 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.38 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.38 min
IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 7.50 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 15.10 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 22.61 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 30.17 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 37.90 sec
IALSRecommenderLine

In [14]:
tuning_params_2 = {
    "alpha":(0,1),
    "beta":(0,1)
}

In [15]:
def BO_function_2(alpha,beta):
    MAP = 0
    k = 5
    cutoff=10
    for i in range(len(similarity_recommenders)):
        recommenders[i].fit(alpha=alpha,beta=beta)
        result_df, _ = evaluator_list[i].evaluateRecommender(recommenders[i])
        MAP += result_df.loc[cutoff]["MAP"]
        
    return MAP/k

In [16]:
optimizer_2 = BayesianOptimization(f=BO_function_2,
                                pbounds=tuning_params_2,
                                verbose=5,
                                random_state=5)

optimizer_2.maximize(init_points=150,n_iter=100)



|   iter    |  target   |   alpha   |   beta    |
-------------------------------------------------
|  1        |  0.2404   |  0.222    |  0.8707   |
|  2        |  0.2398   |  0.2067   |  0.9186   |
|  3        |  0.2465   |  0.4884   |  0.6117   |
|  4        |  0.2487   |  0.7659   |  0.5184   |
|  5        |  0.2489   |  0.2968   |  0.1877   |
|  6        |  0.237    |  0.08074  |  0.7384   |
|  7        |  0.2492   |  0.4413   |  0.1583   |
|  8        |  0.2493   |  0.8799   |  0.2741   |
|  9        |  0.2486   |  0.4142   |  0.2961   |
|  10       |  0.2477   |  0.6288   |  0.5798   |
|  11       |  0.2493   |  0.5999   |  0.2658   |
|  12       |  0.2477   |  0.2847   |  0.2536   |
|  13       |  0.2493   |  0.3276   |  0.1442   |
|  14       |  0.2387   |  0.1656   |  0.9639   |
|  15       |  0.2489   |  0.9602   |  0.1884   |
|  16       |  0.2373   |  0.02431  |  0.2046   |
|  17       |  0.247    |  0.6998   |  0.7795   |
|  18       |  0.2349   |  0.02293  |  0.5777   |


|  100      |  0.2492   |  0.2456   |  0.07127  |
|  101      |  0.2476   |  0.3477   |  0.01285  |
|  102      |  0.2492   |  0.1656   |  0.06037  |
|  103      |  0.2465   |  0.2782   |  0.3483   |
|  104      |  0.2463   |  0.5912   |  0.775    |
|  105      |  0.2491   |  0.6248   |  0.1597   |
|  106      |  0.2481   |  0.8938   |  0.7306   |
|  107      |  0.2464   |  0.4978   |  0.6436   |


KeyboardInterrupt: 

In [17]:
import json
with open("tuning_KFOLD/similarity_merging_SLIM_RP3/" + recommenders[0].RECOMMENDER_NAME + "2_logs.json", 'w') as json_file:
    json.dump(optimizer_2.max, json_file)